In [19]:
import pandas as pd
import numpy as np
import re
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
df = pd.read_pickle("df_A.pkl")

In [21]:
df.shape

(135475, 4)

In [22]:
df.head()

,date,file,item,body
0,20100526,0000950130,Item 3,Notice of Delisting or Failure to Satisfy a Co...
1,20030131,0001001171,None,None
2,20030501,0001001171,None,None
3,20030630,0001001171,None,None
4,20030930,0001001171,None,None


In [23]:
df['date'] =  pd.to_datetime(df['date'], format='%Y%m%d')
df.head()

,date,file,item,body
0,2010-05-26,0000950130,Item 3,Notice of Delisting or Failure to Satisfy a Co...
1,2003-01-31,0001001171,None,None
2,2003-05-01,0001001171,None,None
3,2003-06-30,0001001171,None,None
4,2003-09-30,0001001171,None,None


In [24]:
df.dropna(inplace=True)

In [25]:
df.shape

(76388, 4)

In [26]:
df.item = df.item.apply(lambda x: re.sub('\\n', ' ', x))

In [27]:
df.item = df.item.apply(lambda x: x[5:])

In [28]:
df.reset_index(inplace=True)

In [29]:
df.item.value_counts()

2     23451
5     16632
8     11031
7     10666
1     10531
9      2044
3      1109
4       588
12      316
11       12
6         3
08        3
10        2
Name: item, dtype: int64

In [42]:
for i in df.index:
    if len(df['body'][i]) < 200:
        df.drop(index=i, inplace=True)
df.shape

(29228, 5)

In [43]:
df.head()

,index,date,file,item,body
6,14,2013-10-30,0001104659,8,"On October 30, 2013, Broadway Federal Bank, f...."
8,16,2013-11-15,0001104659,8,"On November 18, 2013, Broadway Financial Corpo..."
10,18,2014-02-28,0001104659,8,"On March 4, 2014, the Company issued a press r..."
19,27,2014-09-24,0001104659,8,"On September 24, 2014, Broadway Financial Corp..."
27,35,2015-05-08,0001104659,2,"On May 8, 2015, Broadway Financial Corporation..."


In [44]:
with open('fin_stopwords.txt') as txt:
    fin_stopwords = txt.read().splitlines()
fin_stopwords

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'c

In [45]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

def spacy_tokenizer(text):
    
    # Create our list of punctuation marks
    punctuations = string.punctuation

    # Create our list of stopwords
    nlp = spacy.load('en_core_web_sm')
    stop_words = spacy.lang.en.stop_words.STOP_WORDS

    # Adding words to remove in addition to stopwords
    add_stopwords = fin_stopwords
    
    # remove numbers and special characters
    pattern = '[0-9]'
    cleantext = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    cleantext2 = re.sub(pattern, '', cleantext)
    
    # remove html tags
    html = re.compile('<.*?>')
    cleantext3 = re.sub(html, '', cleantext2)
    
    # Creating token object
    doc = nlp(cleantext3)

    entity_list = []
    for token in list(doc.ents):
        entities = str(token).split(' ')
        for word in entities:
            entity_list.append(word)
    tokens = [word for word in doc if word.text not in entity_list]

    # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]

    # Removing stop words
    tokens = [ word for word in tokens if word not in stop_words and word not in punctuations ]
    
    # Removing custom set of terms
    tokens = [ word for word in tokens if word not in add_stopwords ]

    # remove words with less than 3 characters
    tokens = [ word for word in tokens if len(word) > 2 ]

    # return preprocessed list of tokens
    return tokens

In [378]:
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# for i in df_A.index:
#     if i in acquisition_ind:
#         text = df_A.tokens[i]

#         # Create and generate a word cloud image:
#         wordcloud = WordCloud(background_color='white').generate(text)

#         # Display the generated image:
#         plt.figure(figsize=(12,6))
#         plt.imshow(wordcloud, interpolation='bilinear')
#         plt.axis("off")
#         plt.show()
#         print(df_A.date[i])
#         print(df_A.body[i])

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [47]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.7, min_df=0.02, stop_words='english')

In [48]:
dtm = cv.fit_transform(df['body'])

In [257]:
tokens_df = pd.DataFrame(dtm.todense(), index=df_A.index, columns=cv.get_feature_names())
tokens_df['date'] = df_A.date

In [258]:
tokens_df.head()

,accounting,acquisition,act,addition,adopt,aggregate,agilent,agreement,amendment,announce,...,term,termination,thereto,time,trade,varian,vice,wholly,whollyowne,world
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [259]:
tokens_df.set_index(pd.to_datetime(tokens_df['date']), inplace=True)
tokens_df.drop(columns='date', inplace=True)
tokens_df.sort_index(inplace=True)

In [260]:
tokens_df = tokens_df.groupby(pd.Grouper(freq='M')).sum()
tokens_df.sort_index(inplace=True)
tokens_df.head()

,accounting,acquisition,act,addition,adopt,aggregate,agilent,agreement,amendment,announce,...,term,termination,thereto,time,trade,varian,vice,wholly,whollyowne,world
date,,,,,,,,,,,,,,,,,,,,,
2003-02-28,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2003-03-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003-04-30,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003-05-31,0,0,0,0,0,0,3,0,0,2,...,0,0,0,1,0,0,0,0,0,0
2003-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [269]:
acquisition_events = tokens_df[tokens_df['acquisition'] > 0]['acquisition'].index

In [276]:
tokens_df.loc[acquisition_events]

,accounting,acquisition,act,addition,adopt,aggregate,agilent,agreement,amendment,announce,...,term,termination,thereto,time,trade,varian,vice,wholly,whollyowne,world
date,,,,,,,,,,,,,,,,,,,,,
2005-11-30,0,1,0,0,0,0,4,3,2,0,...,0,0,0,0,0,0,0,0,0,0
2009-09-30,0,1,1,0,0,0,4,2,0,0,...,0,0,0,2,0,3,0,0,0,0
2010-01-31,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,3,0,0,0,0
2010-04-30,0,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2010-05-31,0,1,0,0,0,0,13,4,0,0,...,2,0,0,0,0,1,0,4,0,0
2012-05-31,0,2,0,0,0,0,6,2,0,0,...,0,0,0,0,0,0,0,0,1,0
2012-08-31,0,1,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [279]:
repurchase_events = tokens_df[tokens_df['repurchase'] > 0]['repurchase'].index
tokens_df.loc[repurchase_events]

,accounting,acquisition,act,addition,adopt,aggregate,agilent,agreement,amendment,announce,...,term,termination,thereto,time,trade,varian,vice,wholly,whollyowne,world
date,,,,,,,,,,,,,,,,,,,,,
2005-12-31,0,0,1,0,0,0,0,2,0,1,...,0,0,0,0,1,0,0,0,0,0
2006-01-31,1,0,0,0,0,3,0,3,0,0,...,0,0,0,0,5,0,0,0,0,5
2006-09-30,0,0,1,0,1,0,8,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2007-09-30,0,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
2007-12-31,0,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
2008-03-31,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,4,0,0,0,2,4
2008-06-30,0,0,0,0,0,2,7,6,2,0,...,0,0,0,0,4,0,0,0,2,4
2008-09-30,0,0,0,0,0,0,0,5,0,0,...,1,0,1,0,2,0,0,0,0,2
2008-11-30,0,0,0,0,0,0,5,4,0,0,...,0,0,0,0,4,0,0,0,0,4


In [278]:
tokens_df.nlargest(3, 'repurchase')

,accounting,acquisition,act,addition,adopt,aggregate,agilent,agreement,amendment,announce,...,term,termination,thereto,time,trade,varian,vice,wholly,whollyowne,world
date,,,,,,,,,,,,,,,,,,,,,
2009-11-30,0,0,0,0,0,0,2,0,0,0,...,0,1,0,2,1,0,0,0,0,0
2006-01-31,1,0,0,0,0,3,0,3,0,0,...,0,0,0,0,5,0,0,0,0,5
2010-11-30,0,0,2,0,0,0,4,1,0,0,...,0,0,1,0,4,0,0,0,0,4


In [168]:
tfidf_transformer = TfidfTransformer()
tfidf_mat = tfidf_transformer.fit_transform(dtm)

In [169]:
tokens_df = pd.DataFrame(dtm.todense(), index=df_A.index, columns=cv.get_feature_names())
tokens_df['date'] = df_A.date

In [170]:
ngrams_long = (pd.melt(ngrams.reset_index(), 
                       id_vars=['index','date'],
                       value_name='token_count')
                 .query('token_count > 0')
                 .sort_values(['index','date']))

In [171]:
tfidf = pd.DataFrame(tfidf_mat.todense(), index=df_A.index, columns=cv.get_feature_names())
tfidf['date'] = df_A.date

tfidf_long = pd.melt(tfidf.reset_index(), 
                     id_vars=['index','date'], 
                     value_name='tfidf').query('tfidf > 0')

In [172]:
full_df = (ngrams_long.merge(tfidf_long, 
                             on=['index','date','variable'])
                      .set_index('index'))

In [173]:
# The lower the IDF value of a word, the less unique it is to any particular document.
full_df.head(5)

,date,variable,token_count,tfidf
index,,,,
1,2003-02-07,senior,2,1.000000
2,2003-02-21,agilent,1,0.201822
2,2003-02-21,announce,1,0.269909
2,2003-02-21,attach,1,0.193473
2,2003-02-21,company,2,0.406580


In [214]:
top_df = full_df[full_df['tfidf'] > .5]
top_df.shape

(167, 4)

In [215]:
top_df = top_df.drop_duplicates(subset='variable')

In [216]:
top_df.sort_values('tfidf', ascending=False)

,date,variable,token_count,tfidf
index,,,,
1,2003-02-07,senior,2,1.000000
40,2015-01-12,business,6,0.861119
148,2010-07-20,technology,1,0.831974
18,2013-03-20,meeting,4,0.733188
130,2009-09-08,arrangement,1,0.732644
78,2006-01-13,financial,4,0.704289
43,2015-03-18,board,3,0.699935
24,2013-12-12,president,2,0.682155
23,2013-11-20,stock,3,0.675380


In [50]:
from sklearn.decomposition import LatentDirichletAllocation

In [114]:
# Build LDA Model with GridSearch params
lda_model = LatentDirichletAllocation(n_components=20,            # Number of topics
                                      learning_decay=0.5,         
                                      max_iter=50,                # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,            # Random state
                                      batch_size=5000,            # n docs in each learning iter
                                      evaluate_every = -1,        # compute perplexity every n iters, default: Don't
                                      n_jobs = -1)                # Use all available CPU

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=5000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.5,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=50,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [115]:
lda_output = lda_model.fit_transform(dtm)

In [116]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -2694822.665297287
Perplexity:  70.39901979788179
{'batch_size': 5000, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.5, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 50, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 42, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


## GridSearch to Identify Best Model Params

In [111]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30],'learning_decay': [.5, .7]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25, batch_size=5000, learning_method='online')

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 30.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=5000,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='online',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=25,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 total

In [112]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 20}
Best Log Likelihood Score:  -907280.8743475805
Model Perplexity:  69.09605222244672


In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [15, 20, 25]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

## Explore Topic Modeling Outputs

In [59]:
lda_model.components_

array([[1.32500317e+02, 1.00087880e-01, 4.46281560e+02, ...,
        1.00005521e-01, 1.48593308e-01, 2.82127143e-01],
       [1.01251874e-01, 1.01044464e-01, 1.02177670e-01, ...,
        8.26003793e+01, 1.00711144e-01, 1.00239798e-01],
       [1.00265132e-01, 1.01098569e-01, 1.23402348e-01, ...,
        9.92236331e+02, 1.00092944e-01, 1.00065617e-01],
       ...,
       [1.00149672e-01, 1.00168714e-01, 3.52486117e+01, ...,
        1.00469912e-01, 1.00163830e-01, 1.02387754e-01],
       [1.05071252e+02, 3.21707866e+02, 8.33397824e+01, ...,
        1.00272691e-01, 1.00041781e-01, 1.00103911e-01],
       [6.31440541e+01, 1.00025565e-01, 5.18238756e+02, ...,
        1.00065291e-01, 8.90177500e+02, 9.43286264e+02]])

In [118]:
# Number of words/tokens in first topic
len(lda_model.components_[0])

210

In [119]:
single_topic = lda_model.components_[0]

In [120]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 42, 141, 207, 196,  73, 101, 119, 171, 205,  62,  15, 116, 120,
         7,  81, 150,  75, 139,  95,  35,  16, 107,  48,  51, 142, 136,
       152, 146,  30, 208, 128,   1, 114,  56,  89, 143, 195, 209, 204,
        58, 182,  94,  77, 167, 188,  78, 127, 132,  39, 206, 115,  69,
       183,  22, 140, 158, 100, 125, 159,  74, 173, 199,  57, 184,  21,
       103,  61, 111,  53,  82, 164, 194,  65,  49,  29,  19, 190,  86,
        20,  13,  63,  28, 112,  45, 160,   3,  24,  85, 113,  70, 155,
       169,  80,  54, 126, 105, 178,  76, 162, 108,  17,  92, 135, 138,
       166, 134,  27, 200,  55, 198,  23,  11, 102, 192, 123,  79,  18,
       165,  10, 133, 187,  46,  64,  91,  66, 172, 104,  68, 121,  93,
        84,  60,  12, 179, 118,  97,  71,  40, 185,  44, 109, 177, 168,
        34, 174,   5,  72,  52,  90,   2, 161, 106,  25,  87, 151, 170,
        96,  88,   8, 147, 197,   0, 189, 137,  59,   4,  14, 163,  98,
       144,  41,  31,  32,  67, 193, 117,  26, 202,  47,  43, 17

In [121]:
# Word least representative of this topic
single_topic[42]

0.05000000000889091

In [122]:
# Word most representative of this topic
single_topic[186]

8506.332378372475

In [123]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([154, 110,  38, 124, 129, 145,  37,  36, 191, 186], dtype=int64)

In [124]:
top_word_indices = single_topic.argsort()[-10:]

In [125]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

purchase
market
companys
offering
option
price
company
common
stock
share


In [126]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['additional', 'time', 'par', 'issue', 'outstanding', 'purchase', 'market', 'companys', 'offering', 'option', 'price', 'company', 'common', 'stock', 'share']


THE TOP 15 WORDS FOR TOPIC #1
['herewith', 'attach', 'current', 'form', 'company', 'copy', 'announce', 'furnish', 'report', 'financial', 'exhibit', 'issue', 'result', 'press', 'release']


THE TOP 15 WORDS FOR TOPIC #2
['period', 'update', 'condition', 'item', 'sale', 'report', 'end', 'provide', 'statement', 'business', 'operation', 'include', 'result', 'financial', 'information']


THE TOP 15 WORDS FOR TOPIC #3
['matter', 'result', 'follow', 'public', 'approve', 'statement', 'proposal', 'company', 'proxy', 'companys', 'hold', 'shareholder', 'vote', 'stockholder', 'meeting']


THE TOP 15 WORDS FOR TOPIC #4
['march', 'pursuant', 'announce', 'set', 'amend', 'addition', 'extend', 'limited', 'time', 'date', 'class', 'exchange', 'supplemental', 'supplement', 'offer']


THE TOP 15 WORDS FOR TOPIC #5
['inc

In [127]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(dtm)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [128]:
df_document_topic.shape

(29228, 21)

In [129]:
df.shape

(29228, 4)

In [130]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,1,4273
1,15,2811
2,7,2788
3,11,1992
4,10,1728
5,0,1685
6,6,1569
7,3,1535
8,14,1457
9,13,1423


In [131]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, dtm, cv, mds='tsne')
panel

C:\Users\nqabe\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
1      -10.878068   10.538728       1        1  9.986208
7      -60.884026  -75.591820       2        1  8.371724
11      -2.336119  -53.289074       3        1  7.242692
0      -61.385319  -12.914634       4        1  7.054908
15     -28.737743  126.296997       5        1  6.456378
14       1.066606 -123.552811       6        1  6.106288
9       45.140720   -7.947869       7        1  5.683007
10    -140.795975  -98.530548       8        1  5.395791
5       61.531681  -83.240562       9        1  4.934743
3      -72.563057 -140.396881      10        1  4.730486
17      15.984491   63.578186      11        1  4.540053
6     -160.851624   68.945145      12        1  4.534047
19      85.961830   51.922935      13        1  4.431402
2      -99.020836  109.936348      14        1  3.970779
13    -176.764130   -8.681684      15        1  3.801689
12    -117.189880  -37.420200      16        1  3.223808
18     108.829918  -22.946203      17        1  3.162120
8       47.588097  121.934814      18        1  2.951833
16    -106.467834   30.176889      19        1  2.303559
4      -50.613804   59.328842      20        1  1.118486, topic_info=    Category          Freq         Term         Total  loglift  logprob
167  Default  22509.000000      release  22509.000000  30.0000  30.0000
143  Default  19584.000000        press  19584.000000  29.0000  29.0000
11   Default  16217.000000    agreement  16217.000000  28.0000  28.0000
186  Default   9653.000000        share   9653.000000  27.0000  27.0000
80   Default   9141.000000         form   9141.000000  26.0000  26.0000
..       ...           ...          ...           ...      ...      ...
14   Topic20    116.593761     announce  11799.787036  -0.1239  -4.1071
130  Topic20     19.287614  outstanding   2005.261515  -0.1509  -5.9063
99   Topic20     95.778480        issue  12455.503925  -0.3747  -4.3037
27   Topic20     15.661217         cash   2489.843093  -0.5756  -6.1146
172  Topic20      5.707413      respect   1096.901874  -0.7653  -7.1240

[731 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         3  0.276098  accordance
0         4  0.132936  accordance
0         7  0.022724  accordance
0        11  0.192019  accordance
0        12  0.094305  accordance
...     ...       ...         ...
207      15  0.931002     website
207      20  0.068322     website
208      16  0.999223      wholly
209       7  0.000989  whollyowne
209      16  0.998588  whollyowne

[1214 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 12, 1, 16, 15, 10, 11, 6, 4, 18, 7, 20, 3, 14, 13, 19, 9, 17, 5])

In [133]:
# Show top n keywords for each topic
def show_topics(vectorizer=cv, lda_model=lda_model, n_words=10):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=cv, lda_model=lda_model, n_words=10)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,share,stock,common,company,price,option,offering,companys,market,purchase
Topic 1,release,press,result,issue,exhibit,financial,report,furnish,announce,copy
Topic 2,information,financial,result,include,operation,business,statement,provide,end,report
Topic 3,meeting,stockholder,vote,shareholder,hold,companys,proxy,company,proposal,statement
Topic 4,offer,supplement,supplemental,exchange,class,date,time,limited,extend,addition
Topic 5,form,report,file,current,statement,company,registration,companys,exhibit,include
Topic 6,director,board,company,effective,serve,committee,member,appoint,elect,companys
Topic 7,reference,incorporate,release,exhibit,press,attach,copy,announce,issue,result
Topic 8,registrant,company,expect,relate,announce,result,previously,development,disclose,net
Topic 9,agreement,purchase,company,enter,sale,transaction,asset,acquisition,agree,acquire


In [104]:
df.reset_index(drop=True, inplace=True)
df.drop(columns='index', inplace=True)
df.head()

,date,file,item,body
0,2013-10-30,0001104659,8,"On October 30, 2013, Broadway Federal Bank, f...."
1,2013-11-15,0001104659,8,"On November 18, 2013, Broadway Financial Corpo..."
2,2014-02-28,0001104659,8,"On March 4, 2014, the Company issued a press r..."
3,2014-09-24,0001104659,8,"On September 24, 2014, Broadway Financial Corp..."
4,2015-05-08,0001104659,2,"On May 8, 2015, Broadway Financial Corporation..."


In [ ]:
# df['tokens'] = df['body'].apply(spacy_tokenizer)
# df.head()

In [134]:
df_topics = df_document_topic.reset_index(drop=True)
df_topics.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
0,0.0,0.00,0.0,0.0,0.00,0.15,0.0,0.0,0.0,0.00,...,0.0,0.32,0.0,0.00,0.30,0.0,0.18,0.00,0.0,12
1,0.0,0.00,0.0,0.0,0.06,0.13,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.00,0.69,0.0,0.07,0.00,0.0,15
2,0.0,0.00,0.0,0.2,0.08,0.07,0.0,0.0,0.0,0.13,...,0.0,0.00,0.0,0.05,0.26,0.0,0.00,0.18,0.0,15
3,0.0,0.33,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.00,0.29,0.0,0.00,0.00,0.0,1
4,0.0,0.38,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.00,0.56,0.0,0.00,0.00,0.0,15


In [135]:
df_combined = pd.merge(df, df_topics, left_index=True, right_index=True)

In [136]:
df_combined.head()

,date,file,item,body,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
0,2013-10-30,0001104659,8,"On October 30, 2013, Broadway Federal Bank, f....",0.0,0.00,0.0,0.0,0.00,0.15,...,0.0,0.32,0.0,0.00,0.30,0.0,0.18,0.00,0.0,12
1,2013-11-15,0001104659,8,"On November 18, 2013, Broadway Financial Corpo...",0.0,0.00,0.0,0.0,0.06,0.13,...,0.0,0.00,0.0,0.00,0.69,0.0,0.07,0.00,0.0,15
2,2014-02-28,0001104659,8,"On March 4, 2014, the Company issued a press r...",0.0,0.00,0.0,0.2,0.08,0.07,...,0.0,0.00,0.0,0.05,0.26,0.0,0.00,0.18,0.0,15
3,2014-09-24,0001104659,8,"On September 24, 2014, Broadway Financial Corp...",0.0,0.33,0.0,0.0,0.00,0.00,...,0.0,0.00,0.0,0.00,0.29,0.0,0.00,0.00,0.0,1
4,2015-05-08,0001104659,2,"On May 8, 2015, Broadway Financial Corporation...",0.0,0.38,0.0,0.0,0.00,0.00,...,0.0,0.00,0.0,0.00,0.56,0.0,0.00,0.00,0.0,15


In [163]:
topic_names_list = list(df_combined.columns[4:-1])

In [155]:
topic_numbers = {0:'Stock Issuance', 1:'Annaul Fin-Results', 2:'Operations Event', 3:'Annual Shareholder Meeting', 4:'Debt Event', 5:'Financial Reporting Event',
                 6:'Board Member Apointment', 7:'Quarterly Fin-Results', 8:'Change in Operations', 9:'Sale of Assets', 10:'Senior Personnel Change', 
                 11:'Information Disclosure', 12:'Joint Venture and Merger Events', 13:'Conference and Presentation', 14:'Credit Agreement',
                 15:'Press Release Other', 16:'Dividend Event', 17:'Lease and Service Agreement', 18:'Note Issuance', 19:'Senior Executive Compensation'}

In [148]:
df_combined.rename(columns=topic_names, inplace=True)

In [154]:
df_combined.dominant_topic.astype(int)

0        12
1        15
2        15
3         1
4        15
         ..
29223    14
29224     3
29225    10
29226     7
29227    14
Name: dominant_topic, Length: 29228, dtype: int32

In [156]:
df_combined['dominant_topic'] = [topic_numbers[number] for number in df_combined.dominant_topic]

In [159]:
df_combined.sort_values('date')

,date,file,item,body,Stock Issuance,Annaul Fin-Results,Operations Event,Annual Shareholder Meeting,Debt Event,Financial Reporting Event,...,Information Disclosure,Joint Venture and Merger Events,Conference and Presentation,Credit Agreement,Press Release Other,Dividend Event,Lease and Service Agreement,Note Issuance,Senior Executive Compensation,dominant_topic
19176,2002-04-03,0000891020,9,In accordance with General Instruction B.2 of ...,0.00,0.00,0.00,0.00,0.0,0.10,...,0.87,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,Information Disclosure
13160,2002-04-27,0001483096,8,"On April 27, 2012, Chesapeake Midstream Partne...",0.00,0.00,0.00,0.00,0.0,0.00,...,0.00,0.15,0.0,0.00,0.48,0.33,0.00,0.0,0.00,Press Release Other
28433,2002-05-14,0000950135,5,"On May 14, 2002, Brooks Automation, Inc. (“Bro...",0.00,0.00,0.00,0.00,0.0,0.00,...,0.00,0.00,0.0,0.15,0.00,0.26,0.00,0.0,0.00,Change in Operations
7977,2002-06-15,0000950133,5,This Form 8-K 12g-3/A is being filed to correc...,0.00,0.00,0.00,0.09,0.0,0.21,...,0.38,0.16,0.0,0.00,0.00,0.00,0.00,0.0,0.04,Information Disclosure
7978,2002-06-15,0000950133,5,"Effective June 15, 2002, pursuant to an Agreem...",0.36,0.00,0.00,0.08,0.0,0.00,...,0.12,0.20,0.0,0.00,0.00,0.00,0.00,0.0,0.04,Stock Issuance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,2020-03-12,0001558370,7,"On March 12, 2020, APLP Holdings Limited Part...",0.00,0.00,0.31,0.00,0.0,0.00,...,0.42,0.10,0.0,0.15,0.00,0.00,0.00,0.0,0.00,Information Disclosure
9867,2020-03-12,0001305323,5,"On March 12, 2020, Zovio Inc (the “Company”) t...",0.00,0.00,0.00,0.00,0.0,0.30,...,0.00,0.00,0.0,0.00,0.00,0.00,0.23,0.0,0.00,Senior Personnel Change
24440,2020-03-12,0000884144,2,"On March 12, 2020, Asure Software, Inc. (the “...",0.00,0.94,0.00,0.00,0.0,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,Annaul Fin-Results
16345,2020-03-13,0000003545,8,"On March 13, 2020, the Board of Directors of A...",0.00,0.00,0.00,0.00,0.0,0.00,...,0.00,0.00,0.0,0.00,0.00,0.93,0.00,0.0,0.00,Dividend Event


In [164]:
df_combined.to_pickle('df_combined.pkl')

## Visualizing Topic Cluster Results & Attributing Labels

In [141]:
def top_5_per_topic(topic=0):
    
    topic_name = 'Topic{}'.format(topic)
    
    print(topic_name)
    
    for i in df_combined[df_combined['dominant_topic'] == topic].sort_values(topic_name, ascending=False).head().index:
        print(df_combined[df_combined['dominant_topic'] == topic].sort_values(topic_name, ascending=False).head().body[i]+'\n\n')

In [142]:
for i in range(0,20):
    top_5_per_topic(i)

Topic0
On March 9, 2004, Antares Pharma, Inc. (the “Company”) completed a third and final
closing of a private placement of its common stock and warrants to several investors. In this third closing, the Company sold an additional 480,050 shares of its common stock, par value $.01 per share, for an aggregate purchase price of $480,050.
The Company also issued to the investors five-year warrants (the “Warrants”) to purchase an aggregate of 160,014 shares of the Company’s common stock at an exercise price of $1.25 per share. In the three closings, the Company sold an
aggregate of 15,120,000 shares of common stock, issued warrants to purchase an aggregate of 5,039,994 shares of common stock and received total aggregate proceeds of $15,120,000. The Company granted the investors customary demand and piggyback
registration rights with respect to the shares of its common stock issued and that are issuable upon exercise of the Warrants.


On February 20, 2004, Antares Pharma, Inc. (the “Company

In [ ]:
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# for i in df_A.index:
#     if i in acquisition_ind:
#         text = df_A.tokens[i]

#         # Create and generate a word cloud image:
#         wordcloud = WordCloud(background_color='white').generate(text)

#         # Display the generated image:
#         plt.figure(figsize=(12,6))
#         plt.imshow(wordcloud, interpolation='bilinear')
#         plt.axis("off")
#         plt.show()
#         print(df_A.date[i])
#         print(df_A.body[i])